# LoRA

LoRA를 활용한 GPT-2 감성 분석 모델 튜닝

In [1]:
!pip install peft datasets transformers


[notice] A new release of pip is available: 23.3.1 -> 25.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from huggingface_hub import login
login(token="")

In [3]:
import torch
import os
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

In [4]:
# gpu 사용 설정을 위한 device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
# 병렬 토크나이저 경고 방지
os.environ["TOKENIZERS_PARALLELISM"]="false"

In [6]:
# 모델 로드
model_name="gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
base_model.config.pad_token_id = tokenizer.pad_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# LoRA 설정
lora_config = LoraConfig(
    r=8,                                            # LoRA 저랭크 지원
    lora_alpha=32,                                  # LoRA 가중치 스케일링 파라미터 
    target_modules=["c_attn", "c_fc", "c_proj"],    # LoRA 적용할 계층
    lora_dropout=0.1                                # 
)

In [8]:
# 기반 모델에 LoRA 적용 (+GPU로 이동)
model = get_peft_model(base_model, lora_config).to(device)

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [9]:
# 데이터셋 로드
dataset = load_dataset("imdb")

In [10]:
# 학습 데이터 준비 
# 샘플링 편향 방지
positive_samples = [sam for sam in dataset['train'] if sam['label'] == 1 ][:500]
negative_samples = [sam for sam in dataset['train'] if sam['label'] == 0 ][:500]

# X, y 데이터 구분
train_texts = [sam["text"] for sam in positive_samples + negative_samples]
train_labels = [sam["label"] for sam in positive_samples + negative_samples]

In [11]:
# 검증용 데이터 준비
# 샘플링 편향 방지
positive_evals = [sam for sam in dataset['test'] if sam['label'] == 1 ][:100]
negative_evals = [sam for sam in dataset['test'] if sam['label'] == 0 ][:100]

# X, y 데이터 구분
eval_texts = [sam["text"] for sam in positive_evals + negative_evals]
eval_labels = [sam["label"] for sam in positive_evals + negative_evals]

In [12]:
def preprocess_data(texts, labels):
    encodings = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )
    encodings["labels"] = torch.tensor(labels, dtype=torch.long)
    return encodings

In [13]:
# 토큰화 처리
train_encodings = preprocess_data(train_texts, train_labels)
eval_encodings = preprocess_data(eval_texts, eval_labels)

In [14]:
class IMDBDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {
            key: val[idx]
            for key, val in self.encodings.items()
        }
        
        

In [15]:
train_dataset = IMDBDataset(train_encodings)
eval_dataset = IMDBDataset(eval_encodings)

In [16]:
# 배치를 GPU로 자동 이동신키는 data_collator 함수
def collate_fn(batch):
    batch = {key: torch.stack([item[key] for item in batch]) for key in batch[0]}
    return batch

In [17]:
training_args = TrainingArguments(
    output_dir='./results',         # 모델 저장 경로 
    per_device_train_batch_size=4,  # 훈련 배치 크기 
    per_device_eval_batch_size=4,   # 평가(검증) 배치 크기
    num_train_epochs=5,             # 학습 횟수 (에포크)
    save_steps=100,                 # 저장 주기
    save_total_limit=2,             # 최대 저장 모델 개수
    eval_strategy='epoch',          # 에포크 단위 평가
    logging_dir='./logs',           # 로드 저장 경로
    logging_steps=10,               # 로그 출력 주기
    fp16=True                       # FP16 연산 최적화
)

In [18]:
# 트레이너 설정
trainer = Trainer(
    model=model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    data_collator=collate_fn
)

No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [19]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,0.294900,No log
2,0.437800,No log
3,0.303500,No log
4,0.200900,No log
5,0.244400,No log


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=625, training_loss=0.38570115413665773, metrics={'train_runtime': 1555.8092, 'train_samples_per_second': 3.214, 'train_steps_per_second': 0.402, 'total_flos': 1324603146240000.0, 'train_loss': 0.38570115413665773, 'epoch': 5.0})

In [20]:
def predict_sentiment(text):
    inputs = tokenizer(
        text,
        return_tensor="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to(device)

    with torch.no_grad():
        outputs = model(**input)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=-1).item()
        
    return "긍정" if prediction == 1 else "부정"

In [21]:
text_review = "I enjoyed watching the monie!"

result = predict_sentiment(text_review)
result

TypeError: PreTrainedTokenizerFast._batch_encode_plus() got an unexpected keyword argument 'return_tensor'